# Baseball rules engine VII

May 28, 2024

Got it somewhat working in the previous version, but then it got cluttered with unit tests.

Going to re-implement clean.

## 0. Utilities 

### 0.2 Print status and print options

In [25]:
def print_status(runners):
    print("\nStatus of runners:")
    for name, runner in runners.items():
        print(f" > {name} on {runner.attained_base} occupies {runner.occupied_base}", end = " | ")
        print(f"He is forced to: {runner.f2_base}. He must tag-up at: {runner.tagup_base}")

def print_options():
    print("HOW TO MAKE RuEg GO.\nCall ru.master_do() and pass a list:")
    print(f"  > Tag runner: ['tag r', name = str]")
    print(f"  > Tag base: ['tag b', base = int]")
    print(f"  > Runner occupy a base: ['occupy', name = str, base = int]")
    print(f"  > Change state: ['state', 'zero', fbc' or 'bip' = str]")
    print(f"  > Create runner: ['create', start_base = int]\n")
    print("*******************\n")

## 1. Baserunner class

In [6]:
class Baserunner:
    
    def __init__(self, name, base):
        self.name = name
        
        self.occupied_base = base
        self.attained_base = base
        
        self.f2_base = None
        self.tagup_base = None

***

## 2. Apply and remove Forces
#### 2.1 Apply Forces
- Triggered by user input. 
- Called by Main > change_state() > state_bip()
- Takes in: 
    - **Baserunner object (the Batter)**
        - occupied = attained = 0B
        - f2_base = 1B
    - **The dict of BR's = {name: Baserunner object}**
- *Starting from Batter, identifies Forces and applies to each BR object*

Returns the dict of runners -- runners may be changed.

In [7]:
class ApplyForces:

    def apply_forces(self, batter, runners):
        
        f2bases = {1: batter, 2: False, 3: False, 4: False}
        
        for base in [1, 2, 3]:
        
            for runner in runners.values():
                attained = runner.attained_base
            
                if base == attained and f2bases[base]:
                    f2bases[ base + 1 ] = runner
                    runner.f2_base = base + 1
        
        return runners

#### 2.2 Remove Forces
- Triggered by FBC or fielder tag (base or BR)
- Called by Main > ?(several paths)? > put_out()
- Takes in:
    - **Baserunner object being put OUT**
    - **The dict of BR's = {name: Baserunner object}**
- *Starting from base of runner being put out, removes Forces for all preceding BR (e.g., out = 1B, 2B = remove force)*

Returns the dict of runners -- runners may be changed 

In [8]:
class RemoveForces:
    
    def remove_forces(self, runner_out, runners):
        kill_force_base = runner_out.f2_base ## This is the base the 'runner_out' is forced to
        
        if kill_force_base:
        
            for base in range(kill_force_base, 5):

                for runner in runners.values():

                    if base == runner.f2_base:
                        runner.f2_base = None
            
        return runners

***

## 3. Create new BR

In [9]:
class CreateRunner:

    def create_batter(self, name):
        batter = self.create_runner(name, 0)
        batter.f2_base = 1
        
        return batter
        
        
    def create_runner(self, name, base):
        runner = Baserunner(name, base)
        print(f"Created {name} on {base}B")
        
        return runner

***

## 4. Occupy & attain base

- Pass ['occupy', name = str, target base = int]

In [10]:
class Occupy:
    
    def occupy_attain_base(self, li, runners):
        name = li[1]
        base = li[2]
        runner = runners[name]

        print(f"\n{name} is attempting to occupy {base}B")

        ## Reject if base is too far away
        attained = runner.attained_base
        if abs(attained - base) > 1:
            print(f"{name} has attained {attained} and cannot occupy {base} because it's too far away")
            return runners

        ## Reject if base is occupied
        for br in runners.values():
            if br.occupied_base == base:
                print(f"Sorry, {base}B is occupied")
                return runners

        print(f"Woo hoo! {base}B is free.")

        ## Else, occupy & attain base 
        runner.attained_base = runner.occupied_base = base

        ## Clear taggable bases
        if runner.f2_base == base:
            runner.f2_base = None

        if runner.tagup_base == base:
            runner.tagup_base = None

        return runners

***

## 99. Main

#### User input
- Tag runner out: **['tag r', 'isaac']** ______ len = 2 | str, str
- Tag base out: **['tag b', 1 or 2 or 3]** ______ len = 2 | str, int
- Occupy & attain a base: **['occupy', 'isaac', 1 or 2 or 3 or 4]** ______ len = 3 | str, str, int
- Change state: **['state', 'zero' or 'fbc' or 'bip']** ______ len = 3 | str, str

In [11]:
class RuEg:
    
    def __init__(self):
        self.af = ApplyForces()
        self.rf = RemoveForces()
        self.cr = CreateRunner()
        self.oc = Occupy()
        
        self.state = -1 ## Initial state

        self.names = ['Isaac', 'Jack', 'JD', 'Romo', 'Casey', 'Sam', 'Pasma', 'Bradey', 'Kemper', 'Liam']
        self.runners = {} # {name: object}
        self.batter = None


    ### Commanded functions ###
    
    def master_do(self, li):
        func_map = {'tag r': self.fielder_tags_runner, 'tag b': self.fielder_tags_base,
                   'occupy': self.occupy_attain_base, 'state': self.change_state,
                   'create': self.create_runner}
        
        key = li[0]
        func_map[ key ]( li )  # Call the function associated with the key and pass the rest of the list.
    
    
    def change_state(self, li):
        new_state = li[1]
        
        if new_state == 'zero':
            print("\nState updated to 0: Pre-pitch \n")
            self.state = 0
            
        elif new_state in ['bip', 'fbc']:
            
            self.state_1()
                
            if new_state == 'bip':
                print("\nSub-state updated to: BIP \n")
                self.runners = self.af.apply_forces(self.batter, self.runners)

            elif new_state == 'fbc':
                self.state_fbc()
    
    
    def state_1(self):
        print("\nState updated to 1", end = " | ")
        self.state = 1
        
        ## All BR de-occupy their base
        for name, runner in self.runners.items():
            runner.occupied_base = None

        ## Instantiate a batter at base 0
        name = self.names.pop(0)
        self.batter = self.cr.create_batter(name)
        self.runners[name] = self.batter

        
    def state_fbc(self):
        print("\nSub-state updated to: FBC", end = " | ")

        self.put_out(self.batter)
        
        for runner in self.runners.values():
            runner.tagup_base = runner.attained_base
         
    
    """ TAGGING RUNNER, BASE """
    
    def fielder_tags_runner(self, li):
        name = li[1]
        runner = self.runners[name]

        out_tracer = False

        if not(runner.occupied_base):
            self.put_out(runner)
            out_tracer = True

        print(f"\nFielder tagged {name} for Out = {out_tracer}")
    
    
    def fielder_tags_base(self, li):
        tagged_base = li[1]
        
        for runner in self.runners.values():
            if runner.f2_base == tagged_base or runner.tagup_base == tagged_base:
                self.put_out(runner)
                print(f"Fielder tagged {tagged_base} to put {runner.name} out.")
                return
       
        print(f"Fielder tagged {tagged_base} -- everybody safe")
        

    def occupy_attain_base(self, li):
        if self.runners:
            self.runners = self.oc.occupy_attain_base(li, self.runners)

            
    def create_runner(self, li):
        start_base = li[1]
        name = self.names.pop(0)
        runner = self.cr.create_runner(name, start_base)
        self.runners[name] = runner
    
    
    
    """ PUT OUT """
    
    def put_out(self, runner_out):
        """ Pass the runner object being put out """
                
        self.runners = self.rf.remove_forces(runner_out, self.runners) # Remove forces for preceding BR        
        del self.runners[runner_out.name] # Delete runner from list of runners
        self.names.append(runner_out.name) # Recycle the name

        print(f"{runner_out.name} is Out.\n")
    
    """ UTILITIES """
        

    def print_status(self):
        print_status(self.runners)
        

In [30]:
state_0 = ['state', 'zero']
fbc = ['state', 'fbc']
bip = ['state', 'bip']

runner = 'Isaac'

tag_r = ['tag r', runner]
tag_1 = ['tag b', 1]
tag_2 = ['tag b', 2]
tag_3 = ['tag b', 3]

occupy_i = ['occupy', 'Isaac', 1]
occupy_j = ['occupy', 'Jack', 3]

create3 = ['create', 3]
create2 = ['create', 2]
create1 = ['create', 1]

In [40]:
ru = RuEg()
#print_options()

ru.master_do(create3)
#ru.master_do(create2)
ru.master_do(create1)

ru.print_status()

ru.master_do(state_0)
ru.master_do(bip)
#ru.master_do(fbc)

#ru.print_status()

#ru.master_do(tag_r)
ru.master_do(tag_1)

#ru.master_do(occupy_i)
#ru.master_do(occupy_j)

ru.print_status()

Created Isaac on 3B
Created Jack on 1B

Status of runners:
 > Isaac on 3 occupies 3 | He is forced to: None. He must tag-up at: None
 > Jack on 1 occupies 1 | He is forced to: None. He must tag-up at: None

State updated to 0: Pre-pitch 


State updated to 1 | Created JD on 0B

Sub-state updated to: BIP 

JD is Out.

Fielder tagged 1 to put JD out.

Status of runners:
 > Isaac on 3 occupies None | He is forced to: None. He must tag-up at: None
 > Jack on 1 occupies None | He is forced to: None. He must tag-up at: None


In [6]:
a = 'A'
b = 3902
if type(a) == str:
    print(a, 'is a string')
if type(b) == int:
    print(b, 'is an int')

A is a string
3902 is an int
